In [39]:
import pandas as pd
import os
from rlhfutils.eval_utils import getapfsft
from statistics import mean, stdev

In [2]:
def respdict(indf):
    resdicts = []
    for ind, row in indf.iterrows():
        #print(row)
        tmp = {'completions':[]}
        assert len(row['response'])==4
        nona = True
        for r in range(4):
            inp, outtmp = getapfsft(row['response'][r])
            tmp['completions'].append({'response':outtmp})
            nona = nona and (outtmp!=None)
        if nona: 
            tmp['instruction'] = inp
            resdicts.append(tmp)
    return pd.DataFrame(resdicts)

In [3]:
filenames = {
    '../../outputs/ultragen/generated_ultraorig.jsonl': 'ultraorig',
    '../../outputs/ultragen/generated_ultra25.jsonl': 'u25',
    '../../outputs/ultragen/generated_ultra50.jsonl': 'u50',
    '../../outputs/ultragen/generated_ultra75.jsonl': 'u75',
    '../../outputs/ultragen/generated_ultra100.jsonl': 'u100',
    '../../outputs/ultragen/generated_ultra200.jsonl': 'u200',
    '../../outputs/ultragen/generated_ultra400.jsonl': 'u400',
    '../../outputs/ultragen/generated_ultra750.jsonl': 'u750',
    '../../outputs/ultragen/generated_ultra975.jsonl': 'u975',
}

In [4]:
# process outputs in prep for ultrafeedback rating usage
for f in filenames.keys():
    outfile = pd.read_json(f, orient='records', lines=True)
    processed = respdict(outfile)
    processed.loc[:50].to_json('../../data/ultraeval_processed/'+filenames[f]+".jsonl", orient='records', lines=True)

In [51]:
processed.loc[:2].to_json('../../data/ultraeval_processed/'+"minitest"+".jsonl", orient='records', lines=True)

In [5]:
ind = 20
print(processed.loc[ind]['instruction'])
print(processed.loc[ind]['completions'][0])

Teacher: In this task you need to give wrong reasons to justify the pronoun coreference relations. Each of the provided inputs contains a sentence with a target pronoun and and a question about how to justify the coreference between a noun phrase and the target pronoun. Good practices involve (1) writing a sentence that gives no reason but relates to the content (e.g., mention the noun phrases that appear in the original sentence); (2) writing a reason that does not justify the relation to the correct noun phrases but another wrong ones.
Teacher: Now, understand the problem? If you are still confused, see the following example:
Sentence: The city councilmen refused the demonstrators a permit because they feared violence. 
 Question: Why does the 'they' refer to the city councilmen?
Solution: Because of the city's history of racial discrimination.
Reason: This is a wrong reason since the reason is descibing issue that does not relates to who will fear the violence.

Now, solve this inst

In [22]:
import os
from rlhfutils.eval_utils import proctmp, count_words

In [23]:
def tulu_to_qa(resps):
    qlist = []
    rlist = []
    for r in resps:
        tmp = r[7:]
        q, a = tmp.split("\n<assistant>\n")
        qlist.append(q)
        rlist.append(a)
    return qlist, rlist

def load_alldfs(base="use_outs/", limit=100, matching=True):
    alldfs = {}
    for f in os.listdir(base):
        print(f)
        if ".jsonl" in f:
            tmp = pd.read_json(base+f, lines=True, orient='records')
            tmp = proctmp(tmp)
            if "<s>" in tmp['response'][0]:
                tmp['response'] = [reconvert(r) for r in tmp['response']]
                tmp['question'] = [reconvert(r) for r in tmp['question']]
            if "### Instruction" in tmp['response'][0]:
                resps = [getapfsft(r)[1] for r in tmp['response']]
                qs = [getapfsft(r)[0] for r in tmp['response']]
                tmp['response'] = resps
                tmp['question'] = qs
            if "<user>" in tmp['response'][0]:
                qs, resps = tulu_to_qa(tmp['response'])
                tmp['question'] = qs
                tmp['response'] = resps
            if "Continue the conversation:\n\n" in tmp['question'][0]:
                print('here')
                tmp['question'] = [s.replace("Continue the conversation:\n\n", "") for s in tmp['question']]
            if "Question:" in tmp['response'][0]:
                tmp['response'] = [r[len(q):] for r, q in zip(tmp['response'], tmp['question'])]
            if "Question:" in tmp['question'][0]:
                tmp['question'] = [s[len("Question: "):-1*len("\n\nAnswer: ")] for s in tmp['question']]
            # constrain to only 200 examples for everything
            #tmp = tmp.loc[:199]
            tmp = tmp.dropna()
            tmp['wcnt'] = count_words(tmp, "question")
            tmp['rcnt'] = count_words(tmp, "response")
            #tmp = tmp[tmp['wcnts']<200].reset_index()
            #tmp = tmp[tmp['rcnts']<200].reset_index()
            alldfs[f.replace("generated_", "").replace(".jsonl", "")] = tmp
    
    valid_indices_sets = []
    if matching:
        for df_key in alldfs:
            if df_key=="davinciwebgpt":
                continue
            df = alldfs[df_key]
            valid_indices_for_df = set(df[(df['wcnt'] < 1000) & (df['rcnt'] < 1000)].index)
            valid_indices_sets.append(valid_indices_for_df)
        
        # Step 2: Find intersection of all sets to get indices valid across all dataframes
        common_valid_indices = set.intersection(*valid_indices_sets)
        
        # Step 3: Filter each dataframe using the common valid indices
        for df_key in alldfs:
            if df_key=="davinciwebgpt":
                continue
            alldfs[df_key] = alldfs[df_key].loc[list(common_valid_indices)].reset_index(drop=True)
            alldfs[df_key] = alldfs[df_key].sort_values(by='question').reset_index(drop=True)
            alldfs[df_key] = alldfs[df_key].loc[:limit]
    #alldfs['davinci']=pd.read_json("../outputs/ckpt_generations/generated_davinci.jsonother", lines=True, orient='records')
    #alldfs['davinci'] = alldfs['davinci'].sort_values(by='question').reset_index(drop=True)
    return alldfs

In [24]:
tdf = pd.read_json("../../outputs/policydpocheck/dpobase.jsonl", orient='records', lines=True)

In [31]:
adfs = load_alldfs("../../outputs/policydpocheck/", 200)

dpofollowppo.jsonl
sftbase.jsonl
dpobase.jsonl


In [33]:
adfs['dpobase']

,question,response,wcnt,rcnt
0,You are given a background paragraph that des...,Output: David's school had just started again ...,806,257
1,do the leaves like to eat hyraces?,"As an AI language model, I cannot provide a de...",12,150
2,"A company has 50 employees, and each employee ...",To solve this problem using mathematical princ...,62,257
3,A text is given in Malayalam. Translate it fro...,"എമ്മങ്ങള്‍ അയ്യേന്‍, മീരു മബ്‍dess നേട്ടി\n****",559,46
4,A text is given in Tamil. Translate it from th...,"ప్రధાనమంత్రణంగా, స‌హ‌ాయ‌తంలో ఔషధ్ ఫౌంచించారు అ...",772,257
...,...,...,...,...
186,write a python script checks in a file written...,"To achieve this, you will need to parse the C ...",62,257
187,write an SEO friendly engaging blog on the top...,The Role of AI in the Growth of the Blockchain...,74,257
188,write an seo blog using the keyword 'internshi...,Are you looking to gain valuable work experien...,28,257
189,write me a paypal business strategic managemen...,Case Study: PayPal's Strategic Management\n\nI...,12,257


In [49]:
adfs.keys()

dict_keys(['dpofollowppo', 'sftbase', 'dpobase'])

In [50]:
fs = [
    '../../outputs/apeval/sftbase_dpofollowppo_0.jsonl',
    '../../outputs/apeval/sftbase_dpofollowppo_100.jsonl'
]
nountrain = pd.concat([pd.read_json(f, orient='records', lines=True) for f in fs])

In [51]:
nountrain

,instruction,input,output_1,output_2,annotator,preference
0,You are given a background paragraph that des...,,Output: Sam wanted to play tennis after school...,Output: Sara had been practicing tennis for ma...,gpt4_3,2
1,do the leaves like to eat hyraces?,,"I'm sorry, but as an AI language model, I cann...","I'm not sure what you mean by ""do the leaves l...",davinci003_2,1
2,"A company has 50 employees, and each employee ...",,One way to approach this problem is to use com...,"Firstly, we can assume that all meetings betwe...",gpt4_2,2
3,A text is given in Malayalam. Translate it fro...,,ആരೋഗ്യ അധികരण സമിതിയാക്കിപ്പോഴെയുള്ള അസുഖാമ്യ ...,ആമ്‍സീ അഖിലുള്ള ചിന്തകങ്ങള്‍ ഉത്പുന്നു\n****,gpt4_3,1
4,A text is given in Tamil. Translate it from th...,,గృహ‌చళ్లని మ‌ధ్య ఫార్మ‌షనాలని ఔట్టు ప్ర‌థానాలన...,"ఈ ఐఠహణ లో, అధికారిగా, చొప్పర్ఘ షవమంత్రి ఏళొ ఔఖ...",chatgpt_2,2
...,...,...,...,...,...,...
86,write a python script checks in a file written...,,To check if all calls to `json_object_object_a...,"To check for this behavior, you'll need to wri...",gpt4_5,1
87,write an SEO friendly engaging blog on the top...,,The Role of AI in the Growth of the Blockchain...,The Role of AI in the Growth of the Blockchain...,davinci003_1,2
88,write an seo blog using the keyword 'internshi...,,Are you looking to gain valuable work experien...,If you are looking to gain valuable work exper...,chatgpt_4,1
89,write me a paypal business strategic managemen...,,"Sure, here is an example of a PayPal business ...",Introduction: PayPal is one of the most widely...,chatgpt_3,1


In [52]:
mean(nountrain.preference)

1.3862433862433863